In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
import os

import codecs


from sklearn.feature_selection import SelectKBest, chi2

In [3]:
# training data
dpc_train_N = pd.read_csv('C:/Users/nchandra/OneDrive - National University of Singapore/FYP/BioStatsFYP/PredProtein/311020 FYP/data/DPC_train_N.csv')

dpc_train_P = pd.read_csv('C:/Users/nchandra/OneDrive - National University of Singapore/FYP/BioStatsFYP/PredProtein/311020 FYP/data/DPC_train_P.csv')


dpc_train = pd.concat([dpc_train_P.iloc[:,1:], dpc_train_N.iloc[:,1:]], axis = 0)
dpc_train.reset_index(drop = True, inplace = True)
dpc_train['target'] = dpc_train['target'].astype('category')

#Train test split
dpc_train_class = dpc_train['target']
dpc_X = dpc_train.drop('target', axis = 1)
dpc_y = dpc_train['target']


In [4]:
# independent test data

dpc_test_N = pd.read_csv('C:/Users/nchandra/OneDrive - National University of Singapore/FYP/BioStatsFYP/PredProtein/311020 FYP/data/DPC_ind_N.csv')

dpc_test_P = pd.read_csv('C:/Users/nchandra/OneDrive - National University of Singapore/FYP/BioStatsFYP/PredProtein/311020 FYP/data/DPC_ind_P.csv')

dpc_test = pd.concat([dpc_test_P.iloc[:,1:], dpc_test_N.iloc[:,1:]], axis = 0)
dpc_test.reset_index(drop = True, inplace = True)
dpc_test['target'] = dpc_test['target'].astype('category')

dpc_indX_test = dpc_test.drop('target', axis = 1)
dpc_indy_test = dpc_test['target']

In [ ]:
# Set seed value
random_seed= 42

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(random_seed)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(random_seed)
# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(random_seed)

### Stack DPC

In [5]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()

In [6]:
X = minmax.fit_transform(dpc_X)
X = pd.DataFrame(X, index = dpc_X.index, columns = dpc_X.columns)
y = dpc_y

In [7]:
X.head()

,AA,AC,AD,AE,AF,AG,AH,AI,AK,AL,...,YM,YN,YP,YQ,YR,YS,YT,YV,YW,YY
0,0.212090,0.0,0.334426,0.320902,0.073770,0.479508,0.000000,0.178279,0.180984,0.295976,...,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.071311,0.0,0.0,0.0
1,0.060174,0.0,0.237209,0.151744,0.104651,0.453488,0.160465,0.252907,0.096279,0.139958,...,0.000000,0.606977,0.00000,0.0,0.160465,0.0,0.202326,0.0,0.0,0.0
2,0.091109,0.0,0.239437,0.229754,0.000000,0.228873,0.242958,0.306338,0.291549,0.211908,...,0.362676,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
3,0.274293,0.0,0.120141,0.230565,0.000000,0.229682,0.243816,0.384276,0.097527,0.318985,...,0.363958,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4,0.221533,0.0,0.000000,0.372432,0.154110,0.111301,0.236301,0.297945,0.283562,0.206102,...,0.352740,0.000000,0.30137,0.0,0.000000,0.0,0.148973,0.0,0.0,0.0


In [8]:
X_train = minmax.fit_transform(dpc_X)
X_train = pd.DataFrame(X_train, index  = dpc_X.index, columns = dpc_X.columns)
y_train = dpc_y

In [9]:
X_test = minmax.fit_transform(dpc_indX_test)
X_test = pd.DataFrame(X_test, index = dpc_indX_test.index, columns = dpc_indX_test.columns)
y_test = dpc_indy_test

### Feature Selection for Sequence with large Attribute > 100

In [ ]:
def feat_select(X_train, y_train): 
    
    from sklearn.feature_selection import SelectFromModel
    from numpy import sort

    xgb = XGBClassifier()
    xgb.fit(X_train, y_train)

    thresholds = np.sort(xgb.feature_importances_)[::-1]

    feature_thresh = pd.DataFrame(columns = ['Thresh', 'n', 'Accuracy'])
    columns = list(feature_thresh)
    data = []

    thresh_limit = 1

    for thresh in thresholds:

        #to Stop the loop if the threshold no longer improves

        if thresh >= thresh_limit:
            break

        #select features using threshold
        selection =  SelectFromModel(xgb, threshold = thresh, prefit = True)
        select_X_train = selection.transform(X_train)

        #train model
        selection_model = XGBClassifier()
        selection_model.fit(select_X_train, y_train)

        #evaluate model
        select_X_test = selection.transform(X_test)
        predictions = selection_model.predict(select_X_test)
        accuracy = accuracy_score(y_test, predictions)


        #print("Thresh = %.3f, n = %d, Accuracy = %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

        thresh_limit = thresh

        values = [thresh, select_X_train.shape[1], accuracy*100]
        zipped = zip(columns, values)
        values_dict = dict(zipped)
        data.append(values_dict)

    feature_thresh = feature_thresh.append(data, True)

    best_accuracy = feature_thresh[feature_thresh['Accuracy'] == feature_thresh['Accuracy'].max()]
    best_threshold = best_accuracy.Thresh.min()

    # transform training data based on the selected feature importances
    best_thresh = SelectFromModel(xgb, threshold = best_threshold, prefit = True)
    
    X_train_best = pd.DataFrame(best_thresh.transform(X_train))
    X_test_best = pd.DataFrame(best_thresh.transform(X_test))

    return X_train_best, X_test_best


In [ ]:
if X_train.shape[1] > 100:
    X_train, X_test = feat_select(X_train, y_train)

## Begin Stack

In [10]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    #level0.append(('lr', LogisticRegression()))
    level0.append(('knn', KNeighborsClassifier()))
    #level0.append(('cart', DecisionTreeClassifier()))
    level0.append(('lsvm', LinearSVC()))
    level0.append(('svm', SVC()))
    level0.append(('et', ExtraTreesClassifier()))
    level0.append(('rf', RandomForestClassifier()))
    #level0.append(('bayes', GaussianNB()))
    level0.append(('xgb', XGBClassifier()))
    
    # define meta learner model
    level1 = LogisticRegression()
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [11]:
def get_models():
    models = dict()
    #models['lr'] = LogisticRegression()
    models['knn'] = KNeighborsClassifier()
    #models['cart'] = DecisionTreeClassifier()
    models['lsvm'] = LinearSVC(C=15.0, dual=False, 
                               loss='squared_hinge', 
                               penalty='l2', tol=0.01)
    models['et'] = ExtraTreesClassifier(bootstrap=False, 
                     criterion = 'entropy', 
                     max_features=0.35000000000000003, 
                     min_samples_leaf=18, 
                     min_samples_split=9, 
                     n_estimators=100)
    
    models['rf'] = RandomForestClassifier()
    models['svm'] = SVC()
    #models['bayes'] = GaussianNB()
    models['xgb'] = XGBClassifier()
    models['stacking'] = get_stacking()
    return models

In [12]:
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, 
                             scoring='roc_auc', 
                             cv=cv, 
                             n_jobs=-1, 
                             error_score='raise')
    return scores

In [13]:
# get the models to evaluate
models = get_models()

In [14]:
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X_train, y_train)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>knn 0.878 (0.054)
>lsvm 0.924 (0.038)
>et 0.916 (0.038)
>rf 0.927 (0.032)
>svm 0.948 (0.038)
>xgb 0.915 (0.038)
>stacking 0.952 (0.027)


In [15]:
# Result on Test Set
results, names = list(), list()
for name, model in models.items():
    mod = model.fit(X_train, y_train)
    y_pred = mod.predict(X_test)
    
    fpr,tpr,thresholds = metrics.roc_curve(y_test, 
                                           y_pred, 
                                           pos_label = 1)
    auc = metrics.auc(fpr,tpr)
    
    results.append(auc)
    names.append(name)
    print('>%s %.4f' % (name, auc))

>knn 0.7059
>lsvm 0.7059
>et 0.5882
>rf 0.6176
>svm 0.6029
>xgb 0.6765
>stacking 0.7353


In [16]:
# Apply Stack model
stackmodel = models['stacking'].fit(X_train,y_train)

In [17]:
y_pred = stackmodel.predict(X_test)

In [18]:
fpr, tpr, thresholds  = metrics.roc_curve(y_test, y_pred, pos_label=1)
print("AUC:",metrics.auc(fpr, tpr) )

AUC: 0.7352941176470588
